In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import sys
from skimage import io, color, filters, feature, restoration

In [4]:
import sys
sys.path.append("..")

In [6]:
from src.standardizer import Standardizer
from src.imageData_generator import ImageGenerator

In [11]:
print('Loading model')
model_path = '../models/models/CNN_E100_Batch10_Filters64_Neurons64_Actrelu_Layers_3.h5'

print('Loading resized images')
resized_imgs = glob.glob('data/medium/*')



Loading model
Loading resized images


In [17]:
print('Subset the images that we want, the ones we trained the model on')
img_list = [164, 202, 425, 345, 139, 72, 311, 363, 403, 509, 362, 257, 175, 203, 47, 183, 0, 297, 34, 8, 320, 197, 293, 450, 215, 28, 74]
img_subset = []
for img in resized_imgs:
    img_idx = int(img.split('/')[2].split('_')[0])
    if img_idx in img_list:
        img_subset.append(img)

print('Standardize the images')
standardizer_subset = Standardizer(img_subset, resized_imgs[3])

print('Select the image we want to scrub')
bin_image = standardizer_subset.binarized_images[8]
grey_image = standardizer_subset.greyscale_image_list[8]
img_name = standardizer_subset.image_list[8].split('/')[2].split('.')[0]
print(img_name)

Subset the images that we want, the ones we trained the model on
Standardize the images
Select the image we want to scrub
403_img
